# Art information retrieval

Functions and code to retrieve more information about given entities in the main Knowledge Graphs containing information about artworks

## Install

In [1]:
#!pip install SPARQLWrapper

from SPARQLWrapper import SPARQLWrapper, JSON
import ssl

import csv

import os

#!pip install rdflib

import rdflib
from rdflib.namespace import CSVW, DC, DCAT, DCTERMS, DOAP, FOAF, ODRL2, ORG, OWL, \
                           PROF, PROV, RDF, RDFS, SDO, SH, SKOS, SOSA, SSN, TIME, \
                           VOID, XMLNS, XSD
from rdflib import Namespace
from rdflib import URIRef
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import time

#!pip install pandas
import pandas as pd

#!pip install numpy
import numpy as np


[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip
C:\Users\Sofia Baroncini\AppData\Roaming\Python\Python39\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Defaulting to user installation because normal site-packages is not writeable


In [35]:
from endpoints import *
from functions import *

In [3]:
wikidata_endpoint

'https://query.wikidata.org/bigdata/namespace/wdq/sparql'

## Functions

In [4]:
from functions import sparql_query_setting, fuzz_ratio_mean, filter_values

In [51]:
# access the results generated by the wd_query_details query
def retreive_wd_details(wd_id):
  wd_details_query = """
  SELECT DISTINCT ?rel ?realpropertyLabel ?obj ?objLabel WHERE {
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
  wd:toBeReplaced ?rel ?obj.
  ?realproperty wikibase:directClaim ?rel.

  }

"""
  query = wd_details_query.replace("toBeReplaced", wd_id)
  resDetails = sparl_query_setting(query, wikidata_endpoint)
  res_list = []
  for result in resDetails["results"]["bindings"]:
    rel = result["rel"]["value"].replace("http://www.wikidata.org/prop/direct/", "")
    relLabel = result["realpropertyLabel"]["value"]
    if result["obj"]["value"].startswith("http://www.wikidata.org/entity/"):
      obj = result["obj"]["value"].replace("http://www.wikidata.org/entity/", "")
    elif result["obj"]["value"].endswith(":00Z"):
      obj = split_date(result["obj"]["value"])
      objLabel = split_date(result["objLabel"]["value"])
    else:
      obj = result["obj"]["value"]
    if result["objLabel"]["value"].endswith(":00Z"):
      objLabel = split_date(result["objLabel"]["value"])
    else:
      objLabel = result["objLabel"]["value"]
    candidate = [rel, relLabel,obj, objLabel]
    confirm = input('Do you like this value?'+ str(candidate) + '(y/n):')
    if confirm == 'y':
      res_list.append(candidate)
   # else:
     # no.append(i)
    # res_list.append([rel, relLabel,obj, objLabel])
  return res_list

# Align artworks to Wikidata


In [39]:
# retrieve all the artwork and artist wd id, if already present
art_dict = {}
icon_query_wd_id = """
    PREFIX icon: <https://w3id.org/icon/ontology/>
    PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>

        SELECT DISTINCT ?artwork ?artist ?artistLabel ?titleLabel ?desc ?wd_artwork ?wd_artist WHERE {

        ?obj crm:P65_shows_visual_item ?artwork; rdfs:label ?desc; ^crm:P108_has_produced / crm:P14_carried_out_by ?artist.
        ?artist rdfs:label ?artistLabel.
        ?rec icon:aboutWorkOfArt ?artwork.
        ?obj crm:P102_has_title / rdfs:label ?titleLabel.
        OPTIONAL {?obj owl:sameAs ?wd_artwork

        FILTER(regex(str(?wd_artwork), ('https://www.wikidata.org/'), 'i'))
        }
        OPTIONAL {?artist owl:sameAs ?wd_artist
        FILTER(regex(str(?wd_artist), ('https://www.wikidata.org/'), 'i'))

        }

        } LIMIT 1000

"""

res_wd_id = sparql_query_setting(icon_query_wd_id, icon_endpoint)

for res in res_wd_id["results"]["bindings"]:
    id = res["artwork"]["value"]
    title = res["titleLabel"]["value"]
    artist_id = res["artist"]["value"]
    artist_label = res["artistLabel"]["value"]
    desc = res["desc"]["value"]
    if "wd_artwork" in res:

        wd_artwork = res["wd_artwork"]["value"].replace("https://www.wikidata.org/wiki/", "https://www.wikidata.org/entity/")
    else:
        wd_artwork = ""
    if "wd_artist" in res:
        wd_artist = res["wd_artist"]["value"]
    else:
        wd_artist = ""
        art_dict[id] = {'title': title, 'description': desc, 'artist_id': artist_id, 'artist_label': artist_label, 'wd_artwork': wd_artwork, 'wd_artist': wd_artist}
        print(id, title, artist_label, wd_artwork, wd_artist)

https://w3id.org/icon/data/ART1163 Pulpit Nicola Pisano  
https://w3id.org/icon/data/ART1385 The Children's Bacchanal Michelangelo  
https://w3id.org/icon/data/ART1387 The Dream Michelangelo  
https://w3id.org/icon/data/ART1006 The firmament Fritz Saxl  
https://w3id.org/icon/data/ART1005test The rape of Proserpina Gian Lorenzo Bernini  
https://w3id.org/icon/data/ART1249 A Hunting Scene Piero di Cosimo  
https://w3id.org/icon/data/ART1368 Ultimate project for the Tombs of the Medici (single-wall tombs, though in this version perhaps still destined for two persons) Michelangelo  
https://w3id.org/icon/data/ART1374 Statue of Lorenzo de' Medici Michelangelo  
https://w3id.org/icon/data/ART1375 Statue of Giuliano de' Medici Michelangelo  
https://w3id.org/icon/data/ART1378 Ganymede Michelangelo  
https://w3id.org/icon/data/ART1379 Tityus Michelangelo  
https://w3id.org/icon/data/ART1141 Virgil Benedetto Antelami  
https://w3id.org/icon/data/ART1148 Crucifixion Giotto  
https://w3id.org/ic

### Open dictionary already stored

In [5]:
# if the dictionary is already stored, open it from csv

    
import pandas as pd
art_dict = {}
data = pd.read_csv('dictionaries/artwork_desc.csv')
data['wd_artwork'] = data['wd_artwork'].astype(str)
data['wd_artist'] = data['wd_artist'].astype(str)
dict_data = data.to_dict('records')
dict_data

for d in dict_data:
    art_dict[d['id']] = {'title' : d['title'], 'description' : d['description'], 'artist_id' : d['artist_id'], 'artist_label' : d['artist_label'], 'wd_artwork' : d['wd_artwork'], 'wd_artist' : d['wd_artist']}
print(art_dict)

    
#  {'id': 'https://w3id.org/icon/data/ART1163', 'title': 'Pulpit', 'description': 'Nicola Pisano, Pulpit, 1256-1260, Pisa, Baptistery', 'artist_id': 'https://w3id.org/icon/data/people/nicola-pisano', 'artist_label': 'Nicola Pisano', 'wd_artwork': 'http://www.wikidata.org/entity/Q3925516', 'wd_artist': 'http://www.wikidata.org/entity/Q364413'}
# for row in data.iterrows(): 
 #   print(row[1]) # row[1][1] # row 0: index, [1][0]: art_id

{'https://w3id.org/icon/data/ART1163': {'title': 'Pulpit', 'description': 'Nicola Pisano, Pulpit, 1256-1260, Pisa, Baptistery', 'artist_id': 'https://w3id.org/icon/data/people/nicola-pisano', 'artist_label': 'Nicola Pisano', 'wd_artwork': 'http://www.wikidata.org/entity/Q3925516', 'wd_artist': 'http://www.wikidata.org/entity/Q364413'}, 'https://w3id.org/icon/data/ART1385': {'title': "The Children's Bacchanal", 'description': "Michelangelo, The Children's Bacchanal, 1533, London, The Royal Collection", 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': 'nan', 'wd_artist': 'nan'}, 'https://w3id.org/icon/data/ART1387': {'title': 'The Dream', 'description': 'Michelangelo, The Dream', 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': 'nan', 'wd_artist': 'nan'}, 'https://w3id.org/icon/data/ART1006': {'title': 'The firmament', 'description': 'Fritz Saxl, The firmament', 'artist

In [18]:
# query on Wikidata. Added an input to manually select case by case if the alignment is correct
not_matched = {}
# already_done = []
for item in art_dict.items():
    if item[1]['wd_artwork'] == "" or item[1]['wd_artwork'] == "nan":
        if item[0] not in already_done:
            print(item[0], item[1]['wd_artwork'])
            if "\'" in item[1]['title']:
                title = item[1]['title'].replace("\'", "")
            else:
                title = item[1]['title']
            artist = item[1]['artist_label']
            descr = item[1]['description']
            q_wd_id = """
            SELECT DISTINCT ?artwork ?artworkLabel ?artist ?artistLabel WHERE {
                 SERVICE wikibase:label { bd:serviceParam wikibase:language '[AUTO_LANGUAGE],en'. }

                ?artwork wdt:P170 ?artist; rdfs:label ?artworkLabel.
                ?artist rdfs:label '"""+artist+"""'@en.

                FILTER(regex(str(?artworkLabel), ('"""+title+"""'), 'i'))
                FILTER (lang(?artworkLabel) = 'en')
                } LIMIT 10




            """
            res = sparql_query_setting(q_wd_id, wikidata_endpoint)
            # manipulate the result
            print("res: ", res, type(res))
            if res["results"]["bindings"] == []:
                already_done.append(item[0])
            else:
                for result in res["results"]["bindings"]:
                    wd_id = result["artwork"]["value"]
                    art_label = result["artworkLabel"]["value"]
                    wd_artist = result["artist"]["value"]
                    artist_label = result["artistLabel"]["value"]
                    candidate = [descr, wd_id, art_label, wd_artist, artist_label]
                    confirm = input('Do you like this value?'+ str(candidate) + '(y/n):')
                    if confirm == 'y':
                        art_dict[item[0]]['wd_artwork'] = wd_id
                        art_dict[item[0]]['wd_artist'] = wd_artist
                    else:
                        not_matched[item[0]] = item[1]
                        # del art_dict[item[0]]
                    time.sleep(5)



https://w3id.org/icon/data/ART1149 nan
res:  {'head': {'vars': ['artwork', 'artworkLabel', 'artist', 'artistLabel']}, 'results': {'bindings': [{'artist': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q15792'}, 'artwork': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q20539148'}, 'artworkLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Triptych: the Crucifixion; the Redeemer with Angels; Saint Nicholas; Saint Gregory'}, 'artistLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Duccio di Buoninsegna'}}, {'artist': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q15792'}, 'artwork': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q118591311'}, 'artworkLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'The Crucifixion'}, 'artistLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Duccio di Buoninsegna'}}, {'artist': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q15792'}, 'artwork': {'type': 'uri', 'value': 'http://www.w

res:  {'head': {'vars': ['artwork', 'artworkLabel', 'artist', 'artistLabel']}, 'results': {'bindings': []}} <class 'dict'>
https://w3id.org/icon/data/ART1193 nan
res:  {'head': {'vars': ['artwork', 'artworkLabel', 'artist', 'artistLabel']}, 'results': {'bindings': []}} <class 'dict'>
https://w3id.org/icon/data/ART1016test nan
res:  {'head': {'vars': ['artwork', 'artworkLabel', 'artist', 'artistLabel']}, 'results': {'bindings': []}} <class 'dict'>
https://w3id.org/icon/data/ART1423 nan
res:  {'head': {'vars': ['artwork', 'artworkLabel', 'artist', 'artistLabel']}, 'results': {'bindings': []}} <class 'dict'>
https://w3id.org/icon/data/ART1015test nan
res:  {'head': {'vars': ['artwork', 'artworkLabel', 'artist', 'artistLabel']}, 'results': {'bindings': []}} <class 'dict'>
https://w3id.org/icon/data/ART1471 nan
res:  {'head': {'vars': ['artwork', 'artworkLabel', 'artist', 'artistLabel']}, 'results': {'bindings': []}} <class 'dict'>
https://w3id.org/icon/data/ART1470 nan
res:  {'head': {'var

In [17]:
already_done

['https://w3id.org/icon/data/ART1385',
 'https://w3id.org/icon/data/ART1387',
 'https://w3id.org/icon/data/ART1006',
 'https://w3id.org/icon/data/ART1368',
 'https://w3id.org/icon/data/ART1374',
 'https://w3id.org/icon/data/ART1375',
 'https://w3id.org/icon/data/ART1378',
 'https://w3id.org/icon/data/ART1141',
 'https://w3id.org/icon/data/ART1150',
 'https://w3id.org/icon/data/ART1283',
 'https://w3id.org/icon/data/ART1276',
 'https://w3id.org/icon/data/ART1275',
 'https://w3id.org/icon/data/ART1365',
 'https://w3id.org/icon/data/ART1310',
 'https://w3id.org/icon/data/ART1169',
 'https://w3id.org/icon/data/ART1177',
 'https://w3id.org/icon/data/ART1170',
 'https://w3id.org/icon/data/ART1181',
 'https://w3id.org/icon/data/ART1179',
 'https://w3id.org/icon/data/ART1180',
 'https://w3id.org/icon/data/ART1176',
 'https://w3id.org/icon/data/ART1178',
 'https://w3id.org/icon/data/ART1182',
 'https://w3id.org/icon/data/ART1183',
 'https://w3id.org/icon/data/ART1184',
 'https://w3id.org/icon/d

In [97]:
print(art_dict["https://w3id.org/icon/data/ART1385"])

{'title': "The Children's Bacchanal", 'description': "Michelangelo, The Children's Bacchanal, 1533, London, The Royal Collection", 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': 'http://www.wikidata.org/entity/Q3937804', 'wd_artist': 'http://www.wikidata.org/entity/Q280851'}


In [57]:
print(art_dict)

{'https://w3id.org/icon/data/ART1163': {'title': 'Pulpit', 'description': 'Nicola Pisano, Pulpit, 1256-1260, Pisa, Baptistery', 'artist_id': 'https://w3id.org/icon/data/people/nicola-pisano', 'artist_label': 'Nicola Pisano', 'wd_artwork': 'http://www.wikidata.org/entity/Q3925516', 'wd_artist': 'http://www.wikidata.org/entity/Q364413'}, 'https://w3id.org/icon/data/ART1385': {'title': "The Children's Bacchanal", 'description': "Michelangelo, The Children's Bacchanal, 1533, London, The Royal Collection", 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1387': {'title': 'The Dream', 'description': 'Michelangelo, The Dream', 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1006': {'title': 'The firmament', 'description': 'Fritz Saxl, The firmament', 'artist_id': 'https

In [ ]:
art_dict = {'https://w3id.org/icon/data/ART1256ext': {'title': 'David', 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': 'http://www.wikidata.org/entity/Q179900', 'wd_artist': 'http://www.wikidata.org/entity/Q5592'}, 'https://w3id.org/icon/data/ART1258ext': {'title': 'The Archers', 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1348': {'title': 'The Prophet Ezekiel', 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1350': {'title': 'The Prophet Isaiah', 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1344': {'title': 'Medal of the Three Graces', 'artist_id': 'https://w3id.org/icon/data/people/niccolo-fiorentino', 'artist_label': 'Niccolò Fiorentino', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1247ext': {'title': 'Pulpit', 'artist_id': 'https://w3id.org/icon/data/people/nicola-pisano', 'artist_label': 'Nicola Pisano', 'wd_artwork': 'http://www.wikidata.org/entity/Q3925518', 'wd_artist': 'http://www.wikidata.org/entity/Q364413'}, 'https://w3id.org/icon/data/ART1244ext': {'title': 'Time Carrying off the Truth', 'artist_id': 'https://w3id.org/icon/data/people/nicolas-poussin', 'artist_label': 'Nicolas Poussin', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1341': {'title': 'Matrimonial Allegory', 'artist_id': 'https://w3id.org/icon/data/people/paris-bordone', 'artist_label': 'Paris Bordone', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1221': {'title': 'Madonna di Foligno', 'artist_id': 'https://w3id.org/icon/data/people/raphael', 'artist_label': 'Raphael', 'wd_artwork': 'http://www.wikidata.org/entity/Q2610709', 'wd_artist': 'http://www.wikidata.org/entity/Q5597'}, 'https://w3id.org/icon/data/ART1349': {'title': 'Fortitude', 'artist_id': 'https://w3id.org/icon/data/people/raphael', 'artist_label': 'Raphael', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1245ext': {'title': 'Time Carrying off the Truth', 'artist_id': 'https://w3id.org/icon/data/people/rubens', 'artist_label': 'Rubens', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1381': {'title': 'Prometheus', 'artist_id': 'https://w3id.org/icon/data/people/rubens', 'artist_label': 'Rubens', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1437': {'title': 'Tragic Scene', 'artist_id': 'https://w3id.org/icon/data/people/sebastiano-serlio', 'artist_label': 'Sebastiano Serlio', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1229': {'title': 'Commodus and Crispina as Mars and Venus', 'artist_id': 'https://w3id.org/icon/data/people/victor-texier', 'artist_label': 'Victor Texier', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1230': {'title': 'Roman sarcophagus', 'artist_id': 'https://w3id.org/icon/data/people/victor-texier', 'artist_label': 'Victor Texier', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1231': {'title': 'Throne of Saturn', 'artist_id': 'https://w3id.org/icon/data/people/victor-texier', 'artist_label': 'Victor Texier', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1364': {'title': "Reintegration of Tomb of Lorenzo de' Medici showing the complete decoration of the entablature and the River Gods, but not the fresco in the lunette and the statues in the lateral niches", 'artist_id': 'https://w3id.org/icon/data/people/ae-popp', 'artist_label': 'A.E. Popp', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1365': {'title': "Reintegration of Tomb of Giuliano de' Medici showing the simplified decoration of the entablature, the fresco of the Brazen Serpent in the lunette, and the River Gods, but not the statue in the lunette and the statues in the lateral niches", 'artist_id': 'https://w3id.org/icon/data/people/ae-popp', 'artist_label': 'A.E. Popp', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1288': {'title': 'The Exposure of Luxury', 'artist_id': 'https://w3id.org/icon/data/people/agnolo-bronzino', 'artist_label': 'Agnolo Bronzino', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1285': {'title': 'The abduction of Proserpine', 'artist_id': 'https://w3id.org/icon/data/people/albrecht-durer', 'artist_label': 'Albrecht Dürer', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1363': {'title': 'Madonna with a monkey (B. 42)', 'artist_id': 'https://w3id.org/icon/data/people/albrecht-durer', 'artist_label': 'Albrecht Dürer', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1358': {'title': 'Holy Water Basin', 'artist_id': 'https://w3id.org/icon/data/people/antonio-federighi', 'artist_label': 'Antonio Federighi', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1312': {'title': 'Amor Divino', 'artist_id': 'https://w3id.org/icon/data/people/francesco-barberino', 'artist_label': 'Francesco Barberino', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1279': {'title': 'Time with Roundel', 'artist_id': 'https://w3id.org/icon/data/people/gian-lorenzo-bernini', 'artist_label': 'Gian Lorenzo Bernini', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1280': {'title': 'Time with Obelisk', 'artist_id': 'https://w3id.org/icon/data/people/gian-lorenzo-bernini', 'artist_label': 'Gian Lorenzo Bernini', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1310': {'title': 'Allegory of Chastity', 'artist_id': 'https://w3id.org/icon/data/people/giotto', 'artist_label': 'Giotto', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1334': {'title': 'Fortitude and Chastity', 'artist_id': 'https://w3id.org/icon/data/people/giovanni-pisano', 'artist_label': 'Giovanni Pisano', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1283': {'title': 'The Vindication of Innocence', 'artist_id': 'https://w3id.org/icon/data/people/giovanni-rost', 'artist_label': 'Giovanni Rost', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1284': {'title': 'Flora', 'artist_id': 'https://w3id.org/icon/data/people/giovanni-rost', 'artist_label': 'Giovanni Rost', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1268': {'title': 'Saturn', 'artist_id': 'https://w3id.org/icon/data/people/jacopo-caraglio', 'artist_label': 'Jacopo Caraglio', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1276': {'title': 'The Triumph of Time', 'artist_id': 'https://w3id.org/icon/data/people/jacopo-del-sellaio', 'artist_label': 'Jacopo del Sellaio', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1275': {'title': 'The Triumph of Time', 'artist_id': 'https://w3id.org/icon/data/people/jacopo-pesellino', 'artist_label': 'Jacopo Pesellino', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1346': {'title': 'Sitting man nude (Fr. 103)', 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1352': {'title': 'Side elevation', 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1355': {'title': 'Second project for the Tomb of Julius II', 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1360': {'title': 'Rebellous Slave', 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1255': {'title': 'Hercules at the Cross Roads', 'artist_id': 'https://w3id.org/icon/data/people/niccolo-soggi', 'artist_label': 'Niccolò Soggi', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1289': {'title': 'Phaethon before Helios', 'artist_id': 'https://w3id.org/icon/data/people/nicolas-poussin', 'artist_label': 'Nicolas Poussin', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1290': {'title': 'A Dance to the Music of Time', 'artist_id': 'https://w3id.org/icon/data/people/nicolas-poussin', 'artist_label': 'Nicolas Poussin', 'wd_artwork': 'http://www.wikidata.org/entity/Q2513438', 'wd_artist': 'http://www.wikidata.org/entity/Q41554'}, 'https://w3id.org/icon/data/ART1353': {'title': 'Tomb of Bishop Simone Saltarello', 'artist_id': 'https://w3id.org/icon/data/people/nino-pisano', 'artist_label': 'Nino Pisano', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1335': {'title': 'Nude Truth', 'artist_id': 'https://w3id.org/icon/data/people/opicinus-de-canistris', 'artist_label': 'Opicinus de Canistris', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1314': {'title': 'Blind Cupid', 'artist_id': 'https://w3id.org/icon/data/people/piero-della-francesca', 'artist_label': 'Piero della Francesca', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1249': {'title': 'A Hunting Scene', 'artist_id': 'https://w3id.org/icon/data/people/piero-di-cosimo', 'artist_label': 'Piero di Cosimo', 'wd_artwork': 'http://www.wikidata.org/entity/Q3649298', 'wd_artist': 'http://www.wikidata.org/entity/Q280851'}, 'https://w3id.org/icon/data/ART1250': {'title': 'The Return from the Hunt', 'artist_id': 'https://w3id.org/icon/data/people/piero-di-cosimo', 'artist_label': 'Piero di Cosimo', 'wd_artwork': 'http://www.wikidata.org/entity/Q3937360', 'wd_artist': 'http://www.wikidata.org/entity/Q280851'}, 'https://w3id.org/icon/data/ART1251': {'title': 'Human Life in the Stone Age', 'artist_id': 'https://w3id.org/icon/data/people/piero-di-cosimo', 'artist_label': 'Piero di Cosimo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1252': {'title': 'Battle of Centaurs and Lapiths', 'artist_id': 'https://w3id.org/icon/data/people/piero-di-cosimo', 'artist_label': 'Piero di Cosimo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1253': {'title': 'The Misfortunes of Silenus', 'artist_id': 'https://w3id.org/icon/data/people/piero-di-cosimo', 'artist_label': 'Piero di Cosimo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1254': {'title': 'The Myth of Prometheus', 'artist_id': 'https://w3id.org/icon/data/people/piero-di-cosimo', 'artist_label': 'Piero di Cosimo', 'wd_artwork': 'https://www.wikidata.org/entity/Q32909273', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1361': {'title': 'Two Fettered Apes', 'artist_id': 'https://w3id.org/icon/data/people/pieter-brueghel-the-elder', 'artist_label': 'Pieter Brueghel the Elder', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1329': {'title': 'Sacred and Profane Love', 'artist_id': 'https://w3id.org/icon/data/people/tiziano-vecellio', 'artist_label': 'Tiziano Vecellio', 'wd_artwork': 'https://www.wikidata.org/entity/Q794077', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1338': {'title': "Allegory of the Marquis d'Avalos", 'artist_id': 'https://w3id.org/icon/data/people/tiziano-vecellio', 'artist_label': 'Tiziano Vecellio', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1339': {'title': 'Education of Cupid', 'artist_id': 'https://w3id.org/icon/data/people/tiziano-vecellio', 'artist_label': 'Tiziano Vecellio', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1163': {'title': 'Pulpit', 'artist_id': 'https://w3id.org/icon/data/people/nicola-pisano', 'artist_label': 'Nicola Pisano', 'wd_artwork': 'http://www.wikidata.org/entity/Q3925518', 'wd_artist': 'http://www.wikidata.org/entity/Q364413'}, 'https://w3id.org/icon/data/ART1158': {'title': 'Presentation of Christ', 'artist_id': 'https://w3id.org/icon/data/people/ambrogio-lorenzetti', 'artist_label': 'Ambrogio Lorenzetti', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1161': {'title': 'Annunciation', 'artist_id': 'https://w3id.org/icon/data/people/ambrogio-lorenzetti', 'artist_label': 'Ambrogio Lorenzetti', 'wd_artwork': 'http://www.wikidata.org/entity/Q3618171', 'wd_artist': 'http://www.wikidata.org/entity/Q211562'}, 'https://w3id.org/icon/data/ART1169': {'title': 'Minerva, Mars and Venus', 'artist_id': 'https://w3id.org/icon/data/people/ambrogio-lorenzetti', 'artist_label': 'Ambrogio Lorenzetti', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1191': {'title': 'David', 'artist_id': 'https://w3id.org/icon/data/people/andrea-del-castagno', 'artist_label': 'Andrea del Castagno', 'wd_artwork': 'http://www.wikidata.org/entity/Q3703387', 'wd_artist': 'http://www.wikidata.org/entity/Q240737'}, 'https://w3id.org/icon/data/ART1192': {'title': 'The Condemnation of St. James', 'artist_id': 'https://w3id.org/icon/data/people/andrea-mantegna', 'artist_label': 'Andrea Mantegna', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1190': {'title': 'Scene from the Aeneid', 'artist_id': 'https://w3id.org/icon/data/people/apollonio-di-giovanni', 'artist_label': 'Apollonio di Giovanni', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1141': {'title': 'Virgil', 'artist_id': 'https://w3id.org/icon/data/people/benedetto-antelami', 'artist_label': 'Benedetto Antelami', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1188': {'title': '"Atys-Amorino" (here interpreted as "Time as a Playful Child Throwing Dice")', 'artist_id': 'https://w3id.org/icon/data/people/donatello', 'artist_label': 'Donatello', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1189': {'title': 'The Heart of the Miser', 'artist_id': 'https://w3id.org/icon/data/people/donatello', 'artist_label': 'Donatello', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1177': {'title': "Annunciation of the Virgin's Death", 'artist_id': 'https://w3id.org/icon/data/people/duccio-di-buoninsegna', 'artist_label': 'Duccio di Buoninsegna', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1205': {'title': 'Erato ("Allegory of Music")', 'artist_id': 'https://w3id.org/icon/data/people/filippino-lippi', 'artist_label': 'Filippino Lippi', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1193': {'title': 'Abduction of Helen', 'artist_id': 'https://w3id.org/icon/data/people/florentine-master', 'artist_label': 'Florentine Master', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1198': {'title': 'Bust of a Young Man', 'artist_id': 'https://w3id.org/icon/data/people/florentine-master', 'artist_label': 'Florentine Master', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1373': {'title': 'Reintegration of  the Sepoltura in Testa showing the David (Bargello) in place and the fresco of the Resurrection in the Lunette', 'artist_id': 'https://w3id.org/icon/data/people/ae-popp', 'artist_label': 'A.E. Popp', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1388': {'title': 'Dream of the Doctor', 'artist_id': 'https://w3id.org/icon/data/people/albrecht-durer', 'artist_label': 'Albrecht Dürer', 'wd_artwork': 'http://www.wikidata.org/entity/Q18338507', 'wd_artist': 'http://www.wikidata.org/entity/Q5580'}, 'https://w3id.org/icon/data/ART1407': {'title': 'Planimetrical Construction of Female Figure (L. 38)', 'artist_id': 'https://w3id.org/icon/data/people/albrecht-durer', 'artist_label': 'Albrecht Dürer', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1408': {'title': 'Stereometrical Construction of Male Figure', 'artist_id': 'https://w3id.org/icon/data/people/albrecht-durer', 'artist_label': 'Albrecht Dürer', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1446': {'title': 'Hercules Killing the Stymphalian Birds', 'artist_id': 'https://w3id.org/icon/data/people/albrecht-durer', 'artist_label': 'Albrecht Dürer', 'wd_artwork': 'http://www.wikidata.org/entity/Q3731240', 'wd_artist': 'http://www.wikidata.org/entity/Q5580'}, 'https://w3id.org/icon/data/ART1448': {'title': 'Rape of the Sabine Woman (L. 347)', 'artist_id': 'https://w3id.org/icon/data/people/albrecht-durer', 'artist_label': 'Albrecht Dürer', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1457': {'title': 'The Fall of Man (B. 1)', 'artist_id': 'https://w3id.org/icon/data/people/albrecht-durer', 'artist_label': 'Albrecht Dürer', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1458': {'title': 'The Resurrection (B. 45)', 'artist_id': 'https://w3id.org/icon/data/people/albrecht-durer', 'artist_label': 'Albrecht Dürer', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1459': {'title': 'Sol Iustitiae (B. 79)', 'artist_id': 'https://w3id.org/icon/data/people/albrecht-durer', 'artist_label': 'Albrecht Dürer', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1462': {'title': 'Nude Warrior (L. 351)', 'artist_id': 'https://w3id.org/icon/data/people/albrecht-durer', 'artist_label': 'Albrecht Dürer', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1456': {'title': 'Bacchanal with the Vat (B. 19)', 'artist_id': 'https://w3id.org/icon/data/people/andrea-mantegna', 'artist_label': 'Andrea Mantegna', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1447': {'title': 'Hercules Killing Nessus', 'artist_id': 'https://w3id.org/icon/data/people/antonio-pollaiuolo', 'artist_label': 'Antonio Pollaiuolo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1424': {'title': 'Allegory of Good Counsel', 'artist_id': 'https://w3id.org/icon/data/people/artus-quellinus-the-elder', 'artist_label': 'Artus Quellinus the Elder', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1507': {'title': 'Camera di San Paolo', 'artist_id': 'https://w3id.org/icon/data/people/correggio', 'artist_label': 'Correggio', 'wd_artwork': 'https://www.wikidata.org/entity/Q3651276', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1434': {'title': 'Project for the Façade of San Petronio', 'artist_id': 'https://w3id.org/icon/data/people/francesco-terribilia', 'artist_label': 'Francesco Terribilia', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1391': {'title': 'Scene of Archery', 'artist_id': 'https://w3id.org/icon/data/people/francisco-de-ollanda', 'artist_label': 'Francisco de Ollanda', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1471': {'title': 'Et in Arcadia Ego', 'artist_id': 'https://w3id.org/icon/data/people/georg-wilhelm-kolbe', 'artist_label': 'Georg Wilhelm Kolbe', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1436': {'title': 'Model for the Façade of Florence Cathedral', 'artist_id': 'https://w3id.org/icon/data/people/gherardo-silvani', 'artist_label': 'Gherardo Silvani', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1435': {'title': 'Project for the Façade of San Petronio', 'artist_id': 'https://w3id.org/icon/data/people/giacomo-barozzi-da-vignola', 'artist_label': 'Giacomo Barozzi da Vignola', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1429': {'title': 'Frame', 'artist_id': 'https://w3id.org/icon/data/people/giorgio-vasari', 'artist_label': 'Giorgio Vasari', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1470': {'title': 'Death even in Arcady', 'artist_id': 'https://w3id.org/icon/data/people/giovanni-battista-cipriani', 'artist_label': 'Giovanni Battista Cipriani', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1418': {'title': 'Fortune', 'artist_id': 'https://w3id.org/icon/data/people/giovanni-zacchi', 'artist_label': 'Giovanni Zacchi', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1467': {'title': 'Et in Arcadia Ego', 'artist_id': 'https://w3id.org/icon/data/people/guercino', 'artist_label': 'Guercino', 'wd_artwork': 'http://www.wikidata.org/entity/Q259012', 'wd_artist': 'http://www.wikidata.org/entity/Q334262'}, 'https://w3id.org/icon/data/ART1420': {'title': 'Allegory of Time', 'artist_id': 'https://w3id.org/icon/data/people/hans-holbein-the-younger', 'artist_label': 'Hans Holbein the Younger', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1472': {'title': 'The Tomb', 'artist_id': 'https://w3id.org/icon/data/people/honore-fragonard', 'artist_label': 'Honoré Fragonard', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1404': {'title': 'Madonna', 'artist_id': 'https://w3id.org/icon/data/people/jacopino-di-francesco', 'artist_label': 'Jacopino di Francesco', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1423': {'title': 'Sol-Apollo', 'artist_id': 'https://w3id.org/icon/data/people/jan-collaert-ii', 'artist_label': 'Jan Collaert II', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1368': {'title': 'Ultimate project for the Tombs of the Medici (single-wall tombs, though in this version perhaps still destined for two persons)', 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1374': {'title': "Statue of Lorenzo de' Medici", 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1375': {'title': "Statue of Giuliano de' Medici", 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1378': {'title': 'Ganymede', 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1379': {'title': 'Tityus', 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': 'http://www.wikidata.org/entity/Q17025697', 'wd_artist': 'http://www.wikidata.org/entity/Q5592'}, 'https://w3id.org/icon/data/ART1382': {'title': 'The Fall of Phaethon (Fr. 57)', 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1383': {'title': 'The Fall of Phaethon (Fr. 75)', 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1384': {'title': 'The Fall of Phaethon (Fr. 58)', 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1385': {'title': "The Children's Bacchanal", 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1387': {'title': 'The Dream', 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1392': {'title': 'Clay model for the Group on the Piazza della Signoria', 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1393': {'title': 'Victory', 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': 'http://www.wikidata.org/entity/Q2438217', 'wd_artist': 'http://www.wikidata.org/entity/Q5592'}, 'https://w3id.org/icon/data/ART1468': {'title': 'Et in Arcadia Ego', 'artist_id': 'https://w3id.org/icon/data/people/nicolas-poussin', 'artist_label': 'Nicolas Poussin', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1469': {'title': 'Et in Arcadia Ego', 'artist_id': 'https://w3id.org/icon/data/people/nicolas-poussin', 'artist_label': 'Nicolas Poussin', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1430': {'title': 'Entrance to a Moat', 'artist_id': 'https://w3id.org/icon/data/people/paul-decker', 'artist_label': 'Paul Decker', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1410': {'title': 'Prudence', 'artist_id': 'https://w3id.org/icon/data/people/rossellino-school-of', 'artist_label': 'Rossellino (school of)', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1380': {'title': 'Tityus', 'artist_id': 'https://w3id.org/icon/data/people/tiziano-vecellio', 'artist_label': 'Tiziano Vecellio', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1409': {'title': 'Allegory of Prudence', 'artist_id': 'https://w3id.org/icon/data/people/tiziano-vecellio', 'artist_label': 'Tiziano Vecellio', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1425': {'title': 'Self-Portrait', 'artist_id': 'https://w3id.org/icon/data/people/tiziano-vecellio', 'artist_label': 'Tiziano Vecellio', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1426': {'title': 'Mater Misericordiae', 'artist_id': 'https://w3id.org/icon/data/people/tiziano-vecellio-and-helpers', 'artist_label': 'Tiziano Vecellio (and helpers)', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1405': {'title': 'Constructed Head', 'artist_id': 'https://w3id.org/icon/data/people/villard-de-honnecourt', 'artist_label': 'Villard de Honnecourt', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1504': {'title': 'The Tomb in Arcady', 'artist_id': 'https://w3id.org/icon/data/people/adam-friedrich-oeser', 'artist_label': 'Adam Friedrich Oeser', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1494': {'title': 'Idylle', 'artist_id': 'https://w3id.org/icon/data/people/alfred-stevens', 'artist_label': 'Alfred Stevens', 'wd_artwork': 'http://www.wikidata.org/entity/Q116167855', 'wd_artist': 'http://www.wikidata.org/entity/Q773855'}, 'https://w3id.org/icon/data/ART1486': {'title': 'St. Sebastian', 'artist_id': 'https://w3id.org/icon/data/people/anthony-van-dyck', 'artist_label': 'Anthony van Dyck', 'wd_artwork': 'http://www.wikidata.org/entity/Q62022439', 'wd_artist': 'http://www.wikidata.org/entity/Q150679'}, 'https://w3id.org/icon/data/ART1505': {'title': 'Et in Arcadia Ego', 'artist_id': 'https://w3id.org/icon/data/people/aubrey-beardsley', 'artist_label': 'Aubrey Beardsley', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1495': {'title': 'At the Tomb of Philippe Pot', 'artist_id': 'https://w3id.org/icon/data/people/charles-edouard-de-beaumont', 'artist_label': 'Charles Edouard de Beaumont', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1439': {'title': "Project for the Remodeling of the Casa dei Borghesi'", 'artist_id': 'https://w3id.org/icon/data/people/domenico-beccafumi', 'artist_label': 'Domenico Beccafumi', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1442': {'title': 'Project for the Decoration of a Façade', 'artist_id': 'https://w3id.org/icon/data/people/domenico-beccafumi', 'artist_label': 'Domenico Beccafumi', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1501': {'title': 'The tomb of Philippe Pot', 'artist_id': 'https://w3id.org/icon/data/people/francois-charles-savinien-petit', 'artist_label': 'François Charles Savinien Petit', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1497': {'title': 'The tomb of Philippe Pot', 'artist_id': 'https://w3id.org/icon/data/people/j-m-s-bence', 'artist_label': 'J. M. S. Bence', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1499': {'title': 'The tomb of Philippe Pot', 'artist_id': 'https://w3id.org/icon/data/people/j-m-s-bence', 'artist_label': 'J. M. S. Bence', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1490': {'title': 'Portrait of a Young Man', 'artist_id': 'https://w3id.org/icon/data/people/jan-van-eyck', 'artist_label': 'Jan van Eyck', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1491': {'title': 'Portrait of a Man in a Red Turban', 'artist_id': 'https://w3id.org/icon/data/people/jan-van-eyck', 'artist_label': 'Jan van Eyck', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1496': {'title': 'The tomb of Philippe Pot', 'artist_id': 'https://w3id.org/icon/data/people/jean-jerome-baugean', 'artist_label': 'Jean Jérôme Baugean', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1488': {'title': 'Musical Party at Melton Constable', 'artist_id': 'https://w3id.org/icon/data/people/john-theodore-heins', 'artist_label': 'John Theodore Heins', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1489': {'title': 'Lady and Youth', 'artist_id': 'https://w3id.org/icon/data/people/member-of-the-zacchia-family', 'artist_label': 'Member of the Zacchia Family', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1438': {'title': 'Comic Scene', 'artist_id': 'https://w3id.org/icon/data/people/sebastiano-serlio', 'artist_label': 'Sebastiano Serlio', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1441': {'title': 'Suggestion for the Remodeling of Gothic Palaces', 'artist_id': 'https://w3id.org/icon/data/people/sebastiano-serlio', 'artist_label': 'Sebastiano Serlio', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1479': {'title': 'Perseus Delivering Andromeds', 'artist_id': 'https://w3id.org/icon/data/people/tiziano-vecellio', 'artist_label': 'Tiziano Vecellio', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1487': {'title': 'Frank Matthew Schutz being sick', 'artist_id': 'https://w3id.org/icon/data/people/william-hogarth', 'artist_label': 'William Hogarth', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1194ext': {'title': 'Flora', 'artist_id': 'https://w3id.org/icon/data/people/agnolo-bronzino', 'artist_label': 'Agnolo Bronzino', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1257ext': {'title': 'Melancholia I', 'artist_id': 'https://w3id.org/icon/data/people/albrecht-durer', 'artist_label': 'Albrecht Dürer', 'wd_artwork': 'http://www.wikidata.org/entity/Q1362177', 'wd_artist': 'http://www.wikidata.org/entity/Q5580'}, 'https://w3id.org/icon/data/ART1239ext': {'title': 'The tomb of Philippe Pot', 'artist_id': 'https://w3id.org/icon/data/people/alphonse-louis-duc', 'artist_label': 'Alphonse Louis Duc', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1222': {'title': 'Madonna and the saints', 'artist_id': 'https://w3id.org/icon/data/people/annibale-carracci', 'artist_label': 'Annibale Carracci', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1238ext': {'title': 'St. Sebastian', 'artist_id': 'https://w3id.org/icon/data/people/anthony-van-dyck', 'artist_label': 'Anthony van Dyck', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1201ext': {'title': 'The Combat of Reason and Love', 'artist_id': 'https://w3id.org/icon/data/people/baccio-bandinelli', 'artist_label': 'Baccio Bandinelli', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1226': {'title': 'Madonna of St. Francis', 'artist_id': 'https://w3id.org/icon/data/people/correggio', 'artist_label': 'Correggio', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1359': {'title': 'Allegorical Engraving', 'artist_id': 'https://w3id.org/icon/data/people/cristoforo-robetta', 'artist_label': 'Cristoforo Robetta', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1229ext': {'title': 'Model of the Lantern of Florence Cathedral', 'artist_id': 'https://w3id.org/icon/data/people/filippo-brunelleschi', 'artist_label': 'Filippo Brunelleschi', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1246ext': {'title': 'Pazzi Chapel', 'artist_id': 'https://w3id.org/icon/data/people/filippo-brunelleschi', 'artist_label': 'Filippo Brunelleschi', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1237ext': {'title': 'Les Mois Grotesque, August', 'artist_id': 'https://w3id.org/icon/data/people/flemish-school', 'artist_label': 'Flemish School', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1343': {'title': 'The Three Graces', 'artist_id': 'https://w3id.org/icon/data/people/francesco-vanni', 'artist_label': 'Francesco Vanni', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1225': {'title': "detail including a small replica of Giovanni da Bologna's Rape of the Sabine Women", 'artist_id': 'https://w3id.org/icon/data/people/g-wingendorp', 'artist_label': 'G. Wingendorp', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1281': {'title': 'Time Revealing Truth', 'artist_id': 'https://w3id.org/icon/data/people/gian-lorenzo-bernini', 'artist_label': 'Gian Lorenzo Bernini', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1223': {'title': 'Immaculate Conception', 'artist_id': 'https://w3id.org/icon/data/people/giorgio-vasari', 'artist_label': 'Giorgio Vasari', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1224': {'title': 'Classical Rome', 'artist_id': 'https://w3id.org/icon/data/people/giovanni-paolo-pannini', 'artist_label': 'Giovanni Paolo Pannini', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1224ext': {'title': 'Sol-Apollo', 'artist_id': 'https://w3id.org/icon/data/people/giovanni-stradano', 'artist_label': 'Giovanni Stradano', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1258': {'title': 'Allegory of Alchemy', 'artist_id': 'https://w3id.org/icon/data/people/girolamo-olgiati', 'artist_label': 'Girolamo Olgiati', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1220': {'title': 'Summer', 'artist_id': 'https://w3id.org/icon/data/people/giuseppe-arcimboldo', 'artist_label': 'Giuseppe Arcimboldo', 'wd_artwork': 'http://www.wikidata.org/entity/Q27978844', 'wd_artist': 'http://www.wikidata.org/entity/Q7751'}, 'https://w3id.org/icon/data/ART1219': {'title': 'The ambassadors', 'artist_id': 'https://w3id.org/icon/data/people/hans-holbein-the-younger', 'artist_label': 'Hans Holbein the Younger', 'wd_artwork': 'http://www.wikidata.org/entity/Q1212937', 'wd_artist': 'http://www.wikidata.org/entity/Q48319'}, 'https://w3id.org/icon/data/ART1225ext': {'title': "Afbeelding van't Stadt Huys van Amsterdam", 'artist_id': 'https://w3id.org/icon/data/people/jan-van-campen', 'artist_label': 'Jan van Campen', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1213': {'title': 'Giovanni Arnolfini and his bride', 'artist_id': 'https://w3id.org/icon/data/people/jan-van-eyck', 'artist_label': 'Jan van Eyck', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1217': {'title': 'Assunta', 'artist_id': 'https://w3id.org/icon/data/people/ludovico-cigoli', 'artist_label': 'Ludovico Cigoli', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1206ext': {'title': 'First project for the Tomb of Julius II', 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1210ext': {'title': "Tomb of Lorenzo de' Medici", 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1211ext': {'title': "Tomb of Giuliano de' Medici", 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1212ext': {'title': 'Tombs of the Medici', 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1213ext': {'title': 'Sepoltura in Testa', 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1216ext': {'title': 'Vault frescoes of the Sistine Chapel', 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1251ext': {'title': 'Front elevation, showing the Moses and the two Slaves in the Louvre in place', 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1252ext': {'title': 'Side elevation, showing the Rebellious Slave in place', 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1253ext': {'title': 'Drawing for Medici tombs', 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1254ext': {'title': 'Drawing for Sepoltura in Testa', 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1148': {'title': 'Crucifixion', 'artist_id': 'https://w3id.org/icon/data/people/giotto', 'artist_label': 'Giotto', 'wd_artwork': 'http://www.wikidata.org/entity/Q3698189', 'wd_artist': 'http://www.wikidata.org/entity/Q7814'}, 'https://w3id.org/icon/data/ART1150': {'title': 'Lamentation of Christ', 'artist_id': 'https://w3id.org/icon/data/people/giotto', 'artist_label': 'Giotto', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1164': {'title': 'Birth of the Virgin', 'artist_id': 'https://w3id.org/icon/data/people/giotto', 'artist_label': 'Giotto', 'wd_artwork': 'http://www.wikidata.org/entity/Q3873250', 'wd_artist': 'http://www.wikidata.org/entity/Q7814'}, 'https://w3id.org/icon/data/ART1170': {'title': 'Banquet of Herod', 'artist_id': 'https://w3id.org/icon/data/people/giotto', 'artist_label': 'Giotto', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1113': {'title': 'Presentation of Christ', 'artist_id': 'https://w3id.org/icon/data/people/nicola-pisano', 'artist_label': 'Nicola Pisano', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1114': {'title': 'Fortitude in the Guise of Hercules', 'artist_id': 'https://w3id.org/icon/data/people/nicola-pisano', 'artist_label': 'Nicola Pisano', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1195': {'title': 'The Finding of Vulcan', 'artist_id': 'https://w3id.org/icon/data/people/piero-di-cosimo', 'artist_label': 'Piero di Cosimo', 'wd_artwork': 'http://www.wikidata.org/entity/Q3937804', 'wd_artist': 'http://www.wikidata.org/entity/Q280851'}, 'https://w3id.org/icon/data/ART1196': {'title': 'Vulcan and Aeolus as Teachers of Mankind', 'artist_id': 'https://w3id.org/icon/data/people/piero-di-cosimo', 'artist_label': 'Piero di Cosimo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1197': {'title': 'The Discovery of Honey', 'artist_id': 'https://w3id.org/icon/data/people/piero-di-cosimo', 'artist_label': 'Piero di Cosimo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1208': {'title': 'Parnassus', 'artist_id': 'https://w3id.org/icon/data/people/raphael', 'artist_label': 'Raphael', 'wd_artwork': 'http://www.wikidata.org/entity/Q2715152', 'wd_artist': 'http://www.wikidata.org/entity/Q5597'}, 'https://w3id.org/icon/data/ART1209': {'title': 'School of Ahens', 'artist_id': 'https://w3id.org/icon/data/people/raphael', 'artist_label': 'Raphael', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1210': {'title': 'Feast of Venus', 'artist_id': 'https://w3id.org/icon/data/people/tiziano-vecellio', 'artist_label': 'Tiziano Vecellio', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1152': {'title': 'Choir Chapel of Reims Cathedral, inside view', 'artist_id': 'https://w3id.org/icon/data/people/villard-de-honnecourt', 'artist_label': 'Villard de Honnecourt', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1153': {'title': 'Choir Chapel of Reims Cathedral, outside view', 'artist_id': 'https://w3id.org/icon/data/people/villard-de-honnecourt', 'artist_label': 'Villard de Honnecourt', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1068': {'title': 'The descent into Limbo', 'artist_id': 'https://w3id.org/icon/data/people/agnolo-bronzino', 'artist_label': 'Agnolo Bronzino', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1019': {'title': 'Celestial Map (B. 151)', 'artist_id': 'https://w3id.org/icon/data/people/albrecht-durer', 'artist_label': 'Albrecht Dürer', 'wd_artwork': 'https://www.wikidata.org/entity/Q18339672', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1009': {'title': 'The Ascension of Christ', 'artist_id': 'https://w3id.org/icon/data/people/correggio', 'artist_label': 'Correggio', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1005test': {'title': 'The rape of Proserpina', 'artist_id': 'https://w3id.org/icon/data/people/gian-lorenzo-bernini', 'artist_label': 'Gian Lorenzo Bernini', 'wd_artwork': 'http://www.wikidata.org/entity/Q2352789', 'wd_artist': 'http://www.wikidata.org/entity/Q160538'}, 'https://w3id.org/icon/data/ART1009test': {'title': 'The Annunciation', 'artist_id': 'https://w3id.org/icon/data/people/jan-van-eyck', 'artist_label': 'Jan van Eyck', 'wd_artwork': 'https://www.wikidata.org/entity/Q2734881', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1193ext': {'title': 'The Vindication of Innocence', 'artist_id': 'https://w3id.org/icon/data/people/agnolo-bronzino', 'artist_label': 'Agnolo Bronzino', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1537': {'title': 'Christ descent to Limbo', 'artist_id': 'https://w3id.org/icon/data/people/agnolo-bronzino', 'artist_label': 'Agnolo Bronzino', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1512': {'title': 'The abduction of Europa and other drawings (L456)', 'artist_id': 'https://w3id.org/icon/data/people/albrecht-durer', 'artist_label': 'Albrecht Dürer', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1513': {'title': 'The Large Fortune (B. 77)', 'artist_id': 'https://w3id.org/icon/data/people/albrecht-durer', 'artist_label': 'Albrecht Dürer', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1514': {'title': 'Hercules at the Crossroads (B. 73)', 'artist_id': 'https://w3id.org/icon/data/people/albrecht-durer', 'artist_label': 'Albrecht Dürer', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1516': {'title': 'Apollo Medicus or Aesculapius (L. 181)', 'artist_id': 'https://w3id.org/icon/data/people/albrecht-durer', 'artist_label': 'Albrecht Dürer', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1517': {'title': 'Apollo-Sol and Diana (L.233)', 'artist_id': 'https://w3id.org/icon/data/people/albrecht-durer', 'artist_label': 'Albrecht Dürer', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1519': {'title': 'Promenade (B. 94)', 'artist_id': 'https://w3id.org/icon/data/people/albrecht-durer', 'artist_label': 'Albrecht Dürer', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1177ext': {'title': 'Martyrdom of the Franciscans in Morocco', 'artist_id': 'https://w3id.org/icon/data/people/ambrogio-lorenzetti', 'artist_label': 'Ambrogio Lorenzetti', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1174ext': {'title': 'Project of the statue of Virgil Planned in 1499', 'artist_id': 'https://w3id.org/icon/data/people/andrea-mantegna', 'artist_label': 'Andrea Mantegna', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1194': {'title': 'Hercules fighting the Hydra', 'artist_id': 'https://w3id.org/icon/data/people/antonio-pollaiuolo', 'artist_label': 'Antonio Pollaiuolo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1180ext': {'title': 'The Combat of Lust and Reason', 'artist_id': 'https://w3id.org/icon/data/people/baccio-bandinelli', 'artist_label': 'Baccio Bandinelli', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1149': {'title': 'Crucifixion', 'artist_id': 'https://w3id.org/icon/data/people/duccio-di-buoninsegna', 'artist_label': 'Duccio di Buoninsegna', 'wd_artwork': 'http://www.wikidata.org/entity/Q28016108', 'wd_artist': 'http://www.wikidata.org/entity/Q15792'}, 'https://w3id.org/icon/data/ART1156': {'title': 'Last Supper', 'artist_id': 'https://w3id.org/icon/data/people/duccio-di-buoninsegna', 'artist_label': 'Duccio di Buoninsegna', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1175': {'title': 'Lamentation of Christ', 'artist_id': 'https://w3id.org/icon/data/people/duccio-di-buoninsegna', 'artist_label': 'Duccio di Buoninsegna', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1536': {'title': 'Tomb of Alexander VII', 'artist_id': 'https://w3id.org/icon/data/people/gian-lorenzo-bernini', 'artist_label': 'Gian Lorenzo Bernini', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1192ext': {'title': 'The Triumphs of Fame, Time, and Eternity', 'artist_id': 'https://w3id.org/icon/data/people/jacopo-pesellino', 'artist_label': 'Jacopo Pesellino', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1520': {'title': 'Risen Christ', 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1521': {'title': 'David', 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': 'https://www.wikidata.org/entity/Q179900', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1530': {'title': 'The creation of Adam', 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': 'http://www.wikidata.org/entity/Q500242', 'wd_artist': 'http://www.wikidata.org/entity/Q5592'}, 'https://w3id.org/icon/data/ART1533': {'title': 'Battle of Centaurs and Lapiths', 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1538': {'title': 'Last Judgement', 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1539': {'title': 'Tomb of Julius II', 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1522': {'title': 'Midas Washing at the Source of the Pactolus', 'artist_id': 'https://w3id.org/icon/data/people/nicolas-poussin', 'artist_label': 'Nicolas Poussin', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1532': {'title': 'The Myth of Prometheus', 'artist_id': 'https://w3id.org/icon/data/people/piero-di-cosimo', 'artist_label': 'Piero di Cosimo', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1151ext': {'title': 'Stories of Cupid and Psyche', 'artist_id': 'https://w3id.org/icon/data/people/raphael', 'artist_label': 'Raphael', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1543': {'title': 'Deposition of Christ', 'artist_id': 'https://w3id.org/icon/data/people/raphael', 'artist_label': 'Raphael', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1534': {'title': 'Saturn devouring a Son', 'artist_id': 'https://w3id.org/icon/data/people/rubens', 'artist_label': 'Rubens', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1096': {'title': 'Cross sections of Piers and Moldings in Reims Cathedral', 'artist_id': 'https://w3id.org/icon/data/people/villard-de-honnecourt', 'artist_label': 'Villard de Honnecourt', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1234': {'title': 'Judith', 'artist_id': 'https://w3id.org/icon/data/people/francesco-maffei', 'artist_label': 'Francesco Maffei', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1166': {'title': 'Triumph of Death', 'artist_id': 'https://w3id.org/icon/data/people/francesco-traini', 'artist_label': 'Francesco Traini', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1181': {'title': 'Flight into Egypt', 'artist_id': 'https://w3id.org/icon/data/people/jacquemart-de-hesdin', 'artist_label': 'Jacquemart de Hesdin', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1179': {'title': 'Four Infancy Scenes', 'artist_id': 'https://w3id.org/icon/data/people/jean-bondol', 'artist_label': 'Jean Bondol', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1180': {'title': 'Portrait of Charles V', 'artist_id': 'https://w3id.org/icon/data/people/jean-bondol', 'artist_label': 'Jean Bondol', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1176': {'title': 'Lamentation of Christ', 'artist_id': 'https://w3id.org/icon/data/people/jean-pucelle', 'artist_label': 'Jean Pucelle', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1178': {'title': 'Annunciation', 'artist_id': 'https://w3id.org/icon/data/people/jean-pucelle', 'artist_label': 'Jean Pucelle', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1182': {'title': 'Presentation of Christ', 'artist_id': 'https://w3id.org/icon/data/people/limbourg-brothers', 'artist_label': 'Limbourg brothers', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1200': {'title': 'Cupid Unblinding Himself', 'artist_id': 'https://w3id.org/icon/data/people/lucas-cranach-the-elder', 'artist_label': 'Lucas Cranach the Elder', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1199': {'title': 'St. Luke Portraying the Virgin', 'artist_id': 'https://w3id.org/icon/data/people/maerten-van-heemskerck', 'artist_label': 'Maerten van Heemskerck', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1183': {'title': 'King Charles VI in Conversation with Pierre Salmon', 'artist_id': 'https://w3id.org/icon/data/people/maitre-des-heures-du-marechal-de-boucicaut', 'artist_label': 'Maitre des Heures du Maréchal de Boucicaut', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1184': {'title': 'Annunciation to the Shepherds', 'artist_id': 'https://w3id.org/icon/data/people/maitre-des-heures-du-marechal-de-boucicaut', 'artist_label': 'Maitre des Heures du Maréchal de Boucicaut', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1159': {'title': 'Presentation of Christ', 'artist_id': 'https://w3id.org/icon/data/people/maitre-des-heures-du-marechal-de-boucicaut-workshop', 'artist_label': 'Maitre des Heures du Maréchal de Boucicaut (workshop)', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1207': {'title': 'Parnassus', 'artist_id': 'https://w3id.org/icon/data/people/marcantonio-raimondi', 'artist_label': 'Marcantonio Raimondi', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1186': {'title': 'The Trinity', 'artist_id': 'https://w3id.org/icon/data/people/masaccio', 'artist_label': 'Masaccio', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1187': {'title': 'Madonna', 'artist_id': 'https://w3id.org/icon/data/people/masaccio', 'artist_label': 'Masaccio', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1131': {'title': 'Cupid with Inverted Torch', 'artist_id': 'https://w3id.org/icon/data/people/master-wiligelmus', 'artist_label': 'Master Wiligelmus', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1132': {'title': 'Cupid with Inverted Torch and Ibis', 'artist_id': 'https://w3id.org/icon/data/people/master-wiligelmus', 'artist_label': 'Master Wiligelmus', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1201': {'title': 'The Combat of Lust and Reason', 'artist_id': 'https://w3id.org/icon/data/people/nicolas-beatrizet', 'artist_label': 'Nicolas Béatrizet', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1185': {'title': 'Adoration of the Magi', 'artist_id': 'https://w3id.org/icon/data/people/north-italian-master', 'artist_label': 'North Italian Master', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1160': {'title': 'Birth of the Virgin', 'artist_id': 'https://w3id.org/icon/data/people/pietro-lorenzetti', 'artist_label': 'Pietro Lorenzetti', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1162': {'title': 'Last Supper', 'artist_id': 'https://w3id.org/icon/data/people/pietro-lorenzetti-workshop', 'artist_label': 'Pietro Lorenzetti (workshop)', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1165': {'title': 'Flagellation of Christ', 'artist_id': 'https://w3id.org/icon/data/people/pietro-lorenzetti-workshop', 'artist_label': 'Pietro Lorenzetti (workshop)', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1232': {'title': 'The vision of the Magi', 'artist_id': 'https://w3id.org/icon/data/people/rogier-van-der-weyden', 'artist_label': 'Rogier van der Weyden', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1202': {'title': 'The Birth of Venus', 'artist_id': 'https://w3id.org/icon/data/people/sandro-botticelli', 'artist_label': 'Sandro Botticelli', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1203': {'title': 'The Realm of Venus ("La primavera")', 'artist_id': 'https://w3id.org/icon/data/people/sandro-botticelli', 'artist_label': 'Sandro Botticelli', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1204': {'title': 'The Calumny of Apelles', 'artist_id': 'https://w3id.org/icon/data/people/sandro-botticelli', 'artist_label': 'Sandro Botticelli', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1157': {'title': "Rejection of Joachim's Offering ; presentation of the Virgin", 'artist_id': 'https://w3id.org/icon/data/people/taddeo-gaddi', 'artist_label': 'Taddeo Gaddi', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1006': {'title': 'The firmament', 'artist_id': 'https://w3id.org/icon/data/people/fritz-saxl', 'artist_label': 'Fritz Saxl', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1006test': {'title': 'Story of Laocoon', 'artist_id': 'https://w3id.org/icon/data/people/polydorus', 'artist_label': 'Polydorus', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1048': {'title': 'Mercury', 'artist_id': 'https://w3id.org/icon/data/people/cyriacus-of-ancona', 'artist_label': 'Cyriacus of Ancona', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1016test': {'title': 'Time and Opportunity', 'artist_id': 'https://w3id.org/icon/data/people/david-le-marchand', 'artist_label': 'David Le Marchand', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1065': {'title': 'Triumph of Venus', 'artist_id': 'https://w3id.org/icon/data/people/francesco-cossa', 'artist_label': 'Francesco Cossa', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1011test': {'title': 'Neptune fountain', 'artist_id': 'https://w3id.org/icon/data/people/tommaso-laureti', 'artist_label': 'Tommaso Laureti', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1062': {'title': 'The death of Patroclus', 'artist_id': 'https://w3id.org/icon/data/people/giulio-romano', 'artist_label': 'Giulio Romano', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1001test': {'title': 'Woman Holding a Balance', 'artist_id': 'https://w3id.org/icon/data/people/johannes-vermeer', 'artist_label': 'Johannes Vermeer', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1017test': {'title': 'Allgory of Catholic Faith', 'artist_id': 'https://w3id.org/icon/data/people/johannes-vermeer', 'artist_label': 'Johannes Vermeer', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1002test': {'title': 'Venus and Cupid', 'artist_id': 'https://w3id.org/icon/data/people/lorenzo-lotto', 'artist_label': 'Lorenzo Lotto', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1012test': {'title': 'Quos Ego', 'artist_id': 'https://w3id.org/icon/data/people/marcantonio-raimondi', 'artist_label': 'Marcantonio Raimondi', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1015test': {'title': 'The rape of Cybele', 'artist_id': 'https://w3id.org/icon/data/people/thomas-regnaudin', 'artist_label': 'Thomas Regnaudin', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1515': {'title': 'Apollo and Dafne', 'artist_id': 'https://w3id.org/icon/data/people/albrecht-durer-workshop', 'artist_label': 'Albrecht Dürer (workshop)', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1020test': {'title': 'The journey of the Magi', 'artist_id': 'https://w3id.org/icon/data/people/benozzo-gozzoli', 'artist_label': 'Benozzo Gozzoli', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1535': {'title': 'Saturn devouring a Son', 'artist_id': 'https://w3id.org/icon/data/people/goya', 'artist_label': 'Goya', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1072': {'title': 'S. Andrea', 'artist_id': 'https://w3id.org/icon/data/people/leon-battista-alberti', 'artist_label': 'Leon Battista Alberti', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1181ext': {'title': 'Cupid Stringing His Bow', 'artist_id': 'https://w3id.org/icon/data/people/lysippus', 'artist_label': 'Lysippus', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1098': {'title': 'St. Andrew', 'artist_id': 'https://w3id.org/icon/data/people/master-gilabertus', 'artist_label': 'Master Gilabertus', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1071': {'title': 'Villa Capra or Villa Rotonda', 'artist_id': 'https://w3id.org/icon/data/people/palladio', 'artist_label': 'Palladio', 'wd_artwork': 'https://www.wikidata.org/entity/Q505574', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1190ext': {'title': 'Saturn', 'artist_id': 'https://w3id.org/icon/data/people/rosso-fiorentino', 'artist_label': 'Rosso Fiorentino', 'wd_artwork': '', 'wd_artist': ''}, 'https://w3id.org/icon/data/ART1527': {'title': 'painted niche', 'artist_id': 'https://w3id.org/icon/data/people/taddeo-gaddi', 'artist_label': 'Taddeo Gaddi', 'wd_artwork': '', 'wd_artist': ''}}

## Store in csv

In [23]:
# store dict in csv
first_line = ["id", "title", "description", "artist_id", "artist_label", "wd_artwork", "wd_artist"]
art_lists = []
for tup in art_dict.items(): 
    col1 = tup[0] 
    col2 = tup[1]['title']
    col3 = tup[1]['description']
    col4 = tup[1]['artist_id']
    col5 = tup[1]['artist_label']
    col6 = tup[1]['wd_artwork']
    col7 = tup[1]['wd_artist']
    art_lists.append([col1, col2, col3, col4, col5, col6, col7])

print(art_lists)
save = store_csv("dictionaries/artwork_desc2.csv", first_line, art_lists)     

[['https://w3id.org/icon/data/ART1163', 'Pulpit', 'Nicola Pisano, Pulpit, 1256-1260, Pisa, Baptistery', 'https://w3id.org/icon/data/people/nicola-pisano', 'Nicola Pisano', 'http://www.wikidata.org/entity/Q3925516', 'http://www.wikidata.org/entity/Q364413'], ['https://w3id.org/icon/data/ART1385', "The Children's Bacchanal", "Michelangelo, The Children's Bacchanal, 1533, London, The Royal Collection", 'https://w3id.org/icon/data/people/michelangelo', 'Michelangelo', 'nan', 'nan'], ['https://w3id.org/icon/data/ART1387', 'The Dream', 'Michelangelo, The Dream', 'https://w3id.org/icon/data/people/michelangelo', 'Michelangelo', 'nan', 'nan'], ['https://w3id.org/icon/data/ART1006', 'The firmament', 'Fritz Saxl, The firmament', 'https://w3id.org/icon/data/people/fritz-saxl', 'Fritz Saxl', 'nan', 'nan'], ['https://w3id.org/icon/data/ART1005test', 'The rape of Proserpina', 'Gian Lorenzo Bernini, The rape of Proserpina, 1622, Rome, Galleria Borghese', 'https://w3id.org/icon/data/people/gian-lorenz

# Artwork alignment with Zeri and Arco

We did noti finalize it since artwork title both on Zeri and Arco are written in italian. We leave it for future works. 

In [38]:
zeri_endpoint = "http://data.fondazionezeri.unibo.it/sparql"

In [29]:
# add dict elements for zeri alignment

for key in art_dict: 
    art_dict[key].update({"zeri_artist": ""})

print(art_dict)

{'https://w3id.org/icon/data/ART1163': {'title': 'Pulpit', 'description': 'Nicola Pisano, Pulpit, 1256-1260, Pisa, Baptistery', 'artist_id': 'https://w3id.org/icon/data/people/nicola-pisano', 'artist_label': 'Nicola Pisano', 'wd_artwork': 'http://www.wikidata.org/entity/Q3925516', 'wd_artist': 'http://www.wikidata.org/entity/Q364413', 'zeri_artwork': '', 'zeri_artist': ''}, 'https://w3id.org/icon/data/ART1385': {'title': "The Children's Bacchanal", 'description': "Michelangelo, The Children's Bacchanal, 1533, London, The Royal Collection", 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': 'nan', 'wd_artist': 'nan', 'zeri_artwork': '', 'zeri_artist': ''}, 'https://w3id.org/icon/data/ART1387': {'title': 'The Dream', 'description': 'Michelangelo, The Dream', 'artist_id': 'https://w3id.org/icon/data/people/michelangelo', 'artist_label': 'Michelangelo', 'wd_artwork': 'nan', 'wd_artist': 'nan', 'zeri_artwork': '', 'zeri_artist': ''},

In [30]:
already_done_zeri = []

In [42]:
# query on Wikidata. Added an input to manually select case by case if the alignment is correct
not_matched_zeri = {}
# already_done = []
for item in art_dict.items():
    if item[1]['zeri_artwork'] == "" or item[1]['zeri_artwork'] == "nan":
        if item[0] not in already_done_zeri:
            print(item[0], item[1]['zeri_artwork'])
            if "\'" in item[1]['title']:
                title = item[1]['title'].replace("\'", "")
            else:
                title = item[1]['title']
            artist = item[1]['artist_label']
            if len(artist.split(" ")) > 1: 
                
                name = artist.split(" ")[0]
                surname = artist.split(" ")[1]
            else: 
                name = artist
                surname = artist
                
            descr = item[1]['description']
            zeri_id = """
            PREFIX fabio: <http://purl.org/spar/fabio/>
            PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
            PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            SELECT DISTINCT ?artwork ?artworkLabel ?artist ?artistLabel WHERE {
              ?artwork a fabio:ArtisticWork ;
                        rdfs:label ?artworkLabel;
                        # crm:P4_has_time_span ?timespan;
                        crm:P94i_was_created_by / crm:P14_carried_out_by ?artist. 
              ?artist rdfs:label ?artistLabel.
              FILTER(regex(str(?artistLabel), ('"""+name+"""'), 'i'))
              FILTER(regex(str(?artistLabel), ('"""+surname+"""'), 'i'))
              FILTER(regex(str(?artworkLabel), ('"""+title+"""'), 'i'))

            } LIMIT 10




            """
            res = sparql_query_setting(zeri_id, zeri_endpoint)
            time.sleep(0.5)
            # manipulate the result
            print("res: ", res, type(res))
            if res["results"]["bindings"] == []:
                already_done_zeri.append(item[0])
            else:
                for result in res["results"]["bindings"]:
                    zeri_id = result["artwork"]["value"]
                    art_label = result["artworkLabel"]["value"]
                    zeri_artist = result["artist"]["value"]
                    artist_label = result["artistLabel"]["value"]
                    candidate = [descr, zeri_id, art_label, zeri_artist, artist_label]
                    confirm = input('Do you like this value?'+ str(candidate) + '(y/n):')
                    if confirm == 'y':
                        art_dict[item[0]]['zeri_artwork'] = zeri_id
                        art_dict[item[0]]['zeri_artist'] = zeri_artist
                    else:
                        not_matched_zeri[item[0]] = item[1]
                        # del art_dict[item[0]]
            



https://w3id.org/icon/data/ART1521 
res:  {'head': {'vars': ['artwork', 'artworkLabel', 'artist', 'artistLabel']}, 'results': {'bindings': [{'artwork': {'type': 'uri', 'value': 'https://w3id.org/zericatalog/artwork/16569'}, 'artworkLabel': {'type': 'literal', 'xml:lang': 'en', 'value': 'Artwork "Buonarroti Michelangelo , Creazione di Eva, Ignudi, Distruzione della tribù di Achab, David ammonito da Nathan" , (component)'}, 'artist': {'type': 'uri', 'value': 'https://w3id.org/zericatalog/person/2224/buonarroti-michelangelo'}, 'artistLabel': {'type': 'literal', 'xml:lang': 'it', 'value': 'Buonarroti Michelangelo'}}, {'artwork': {'type': 'uri', 'value': 'https://w3id.org/zericatalog/artwork/16569'}, 'artworkLabel': {'type': 'literal', 'xml:lang': 'it', 'value': 'Opera d\'arte "Buonarroti Michelangelo , Creazione di Eva, Ignudi, Distruzione della tribù di Achab, David ammonito da Nathan" , (elemento d\'insieme)'}, 'artist': {'type': 'uri', 'value': 'https://w3id.org/zericatalog/person/2224/

res:  {'head': {'vars': ['artwork', 'artworkLabel', 'artist', 'artistLabel']}, 'results': {'bindings': []}} <class 'dict'>
https://w3id.org/icon/data/ART1515 
res:  {'head': {'vars': ['artwork', 'artworkLabel', 'artist', 'artistLabel']}, 'results': {'bindings': []}} <class 'dict'>
https://w3id.org/icon/data/ART1418 
res:  {'head': {'vars': ['artwork', 'artworkLabel', 'artist', 'artistLabel']}, 'results': {'bindings': []}} <class 'dict'>
https://w3id.org/icon/data/ART1467 
res:  {'head': {'vars': ['artwork', 'artworkLabel', 'artist', 'artistLabel']}, 'results': {'bindings': []}} <class 'dict'>
https://w3id.org/icon/data/ART1426 
res:  {'head': {'vars': ['artwork', 'artworkLabel', 'artist', 'artistLabel']}, 'results': {'bindings': []}} <class 'dict'>
https://w3id.org/icon/data/ART1288 
res:  {'head': {'vars': ['artwork', 'artworkLabel', 'artist', 'artistLabel']}, 'results': {'bindings': []}} <class 'dict'>
https://w3id.org/icon/data/ART1198 
res:  {'head': {'vars': ['artwork', 'artworkLa

QueryBadFormed: QueryBadFormed: a bad request has been sent to the endpoint, probably the sparql query is bad formed. 

Response:
b"Error 400: Error: \n\n            PREFIX fabio: <http://purl.org/spar/fabio/>\n            PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>\n            PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\n            SELECT DISTINCT ?artwork ?artworkLabel ?artist ?artistLabel WHERE {\n              ?artwork a fabio:ArtisticWork ;\n                        rdfs:label ?artworkLabel;\n                        # crm:P4_has_time_span ?timespan;\n                        crm:P94i_was_created_by / crm:P14_carried_out_by ?artist. \n              ?artist rdfs:label ?artistLabel.\n              FILTER(regex(str(?artistLabel), ('Rossellino'), 'i'))\n              FILTER(regex(str(?artistLabel), ('(school'), 'i'))\n              FILTER(regex(str(?artworkLabel), ('Prudence'), 'i'))\n\n            } LIMIT 10\n\n\n\n\n            \n\rRegex: Pattern exception: java.util.r

In [37]:
zeri_endpoint

NameError: name 'zeri_endpoint' is not defined